In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import TrainingArguments, Trainer
from transformers import BartTokenizer, BartForConditionalGeneration, EarlyStoppingCallback
import pickle

device = torch.device("cuda")
print('GPU:', torch.cuda.get_device_name(0))

import warnings
warnings.filterwarnings('ignore')

GPU: A100-SXM4-40GB


In [2]:
# load model and tokenizer
model_name = "facebook/bart-large-cnn"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

# freeze encoder layers
for param in model.base_model.encoder.parameters():
    param.requires_grad = False

for param in model.base_model.shared.parameters():
    param.requires_grad = False

In [3]:
# load tokenized training/dev data
train_source_tokenized  = torch.load('train_source_tokenized.pt')
train_target_tokenized  = torch.load('train_target_tokenized.pt')
dev_source_tokenized  = torch.load('dev_source_tokenized.pt')
dev_target_tokenized  = torch.load('dev_target_tokenized.pt')

In [4]:
# custom dataset class for data loading
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, source, target):
        self.source = source
        self.target = target

    def __getitem__(self, idx):
        target = {key: val[idx].detach().clone() for key, val in self.target.items()}
        target_input_ids = target['input_ids'][:-1].detach().clone()
        target_attention_mask = target['attention_mask'][:-1].detach().clone()

        labels = target['input_ids'][1:].detach().clone()
        labels[labels[:] == tokenizer.pad_token_id] = -100

        source = {key: val[idx].detach().clone() for key, val in self.source.items()}
        
        item = {'input_ids': source['input_ids'].detach().clone(),
                'attention_mask': source['attention_mask'].detach().clone(),
                'decoder_input_ids': target_input_ids.detach().clone(),
                'decoder_attention_mask': target_attention_mask.detach().clone(),
                'labels': labels}

        return item

    def __len__(self):
        return len(self.source["input_ids"])

train_dataset = CustomDataset(train_source_tokenized, train_target_tokenized)
val_dataset = CustomDataset(dev_source_tokenized, dev_target_tokenized)

In [ ]:
# arguments for Huggingface trainer class
args = TrainingArguments(
    num_train_epochs=5,
    per_device_train_batch_size=10,
    per_device_eval_batch_size=10,
    output_dir="output",
    save_strategy = "no",
    metric_for_best_model = 'eval_loss',
    load_best_model_at_end=True,
    fp16 = True,
    evaluation_strategy = "steps",
    eval_steps=1000,
    learning_rate = 1.97622e-05,
    weight_decay = 0.0298317,
    seed = 21)


trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience = 2)])

trainer.train()

Step,Training Loss,Validation Loss,Runtime,Samples Per Second
1000,1.829500,1.773201,671.241800,148.978000
2000,1.783000,1.738948,671.427600,148.936000
3000,1.761500,1.719797,666.265700,150.090000
4000,1.746800,1.705974,666.166700,150.113000
5000,1.736900,1.695640,666.161400,150.114000
6000,1.639000,1.692161,666.697200,149.993000
7000,1.643300,1.687195,668.212000,149.653000
8000,1.640200,1.681399,668.934100,149.492000
9000,1.637000,1.674053,668.748100,149.533000
10000,1.631100,1.671935,666.417200,150.056000
